In [2]:
import pandas as pd
clientes = pd.read_csv('caso_estudo_clientes.csv', sep=';')
lojas = pd.read_csv('caso_estudo_lojas.csv', sep=';')
pagamentos = pd.read_csv('caso_estudo_pagamentos.csv', sep=';')
produtos = pd.read_csv('caso_estudo_produtos.csv', sep=';')
vendas = pd.read_csv('caso_estudo_vendas.csv', sep=';')

# Limpeza de Dados

A limpeza de dados consiste na preparação dos dados que serão analisados.
Serão buscados neste processo:
- Dados Nulos
- Dados duplicados
- Outliers (fora da faixa/média)
- Ajustar tipos de dados (ex: datas)
- Se necessita unir bases (join) e realizar correspondências

## Dados Nulos 

Primeira situação: Modificar os dados

In [6]:
vendas

,id,id_cliente,id_loja,id_produto,dt_venda
0,1,620,3,2,07/11/2018
1,2,483,1,9,06/01/2018
2,3,976,8,3,10/04/2018
3,4,389,5,6,05/07/2020
4,5,626,7,8,22/07/2018
...,...,...,...,...,...
995,996,641,8,5,27/03/2018
996,997,325,4,1,26/07/2019
997,998,765,6,8,24/03/2018
998,999,217,10,3,23/08/2018


In [7]:
# isnull() - verificar se existem dados nulos no Dataframe
# Retorna True caso algum dado seja nulo, retorna False se não for nulo
clientes.isnull()

,id,nome,sexo,dt_nasc
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
995,False,False,False,False
996,False,False,False,False
997,False,False,False,False
998,False,False,False,False


In [8]:
'''
A junção da função isnull() com sum() retorna quantos dados
nulos existem por coluna
'''
clientes.isnull().sum()

id         0
nome       4
sexo       4
dt_nasc    4
dtype: int64

In [13]:
'''
Revisão de Slicing
Usando loc é possível fatiar linhas (2:5) e colunas ('nome':'sexo')
ao mesmo tempo
'''
clientes.loc[2:5,'nome':'sexo']

,nome,sexo
2,Júlio Azevedo,M
3,Sarah Goncalves,F
4,Gabrielle Martins,F
5,Felipe Almeida,M


In [16]:
'''
T é uma alias (abreviação) para Transpose, e tem como objetivo de
verificar algo em uma linha completa
any() retorna True se alguma coluna da linha for nulo, False caso contrário
'''
clientes.isnull().T.any()

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

In [18]:
'''
A operação anterior foi colocada nos colchetes para o slicing,
retornando apenas as linhas que possuem alguma coluna com dado nulo
'''
clientes[clientes.isnull().T.any()]

,id,nome,sexo,dt_nasc
263,264,NaN,F,7/29/1997
264,265,NaN,M,2/7/1979
268,269,NaN,F,NaN
269,270,NaN,M,NaN
276,277,Brenda Santos,F,NaN
282,283,Amanda Barros,F,NaN
287,288,Victor Ribeiro,NaN,12/17/1974
290,291,Clara Pinto,NaN,12/25/1980
294,295,Tiago Lima,NaN,7/26/1973
301,302,Guilherme Cardoso,NaN,3/9/1998


In [29]:
'''
É necessário verificar se os clientes que possuem dados nulos tem
algum tipo de dependência no dataframe. Se possuir, esses dados não
podem ser apagados.
Neste exemplo, foi realizado uma busca no dataframe vendas pelo id 
do primeiro cliente que possui dados nulos.
'''
vendas[vendas.id_cliente == 264]

,id,id_cliente,id_loja,id_produto,dt_venda
32,33,264,1,3,22/02/2020
815,816,264,3,3,28/01/2019
906,907,264,6,5,21/02/2019


In [7]:
'''
Retorna apenas os índices dos clientes
que possuem pelo menos 1 dado nulo
'''
clientes[clientes.isnull().T.any()].index

Int64Index([263, 264, 268, 269, 276, 282, 287, 290, 294, 301], dtype='int64')

In [9]:
'''
A operação a seguir retorna todas as vendas "vendas[]" em que o
id_cliente de vendas "vendas['id_cliente']" esteja contido "isin()"
na lista de ids de clientes que possuem pelo menos 1 dado nulo
"clientes[clientes.isnull().T.any()].index)"
'''
vendas[vendas['id_cliente'].isin(clientes[clientes.isnull().T.any()].index)]

,id,id_cliente,id_loja,id_produto,dt_venda
32,33,264,1,3,22/02/2020
82,83,301,6,4,14/08/2020
197,198,268,2,1,01/03/2018
296,297,301,4,7,13/06/2019
538,539,276,5,4,26/09/2019
597,598,263,8,6,07/07/2018
815,816,264,3,3,28/01/2019
906,907,264,6,5,21/02/2019
917,918,269,7,10,20/09/2018


In [12]:
#Slicing simples para retornar os clientes que possuem apenas o nome nulo
clientes[clientes.nome.isnull()]

,id,nome,sexo,dt_nasc
263,264,NaN,F,7/29/1997
264,265,NaN,M,2/7/1979
268,269,NaN,F,NaN
269,270,NaN,M,NaN


In [13]:
#Slicing utilizando loc que retorna os mesmos dados do exemplo anterior
'''
Revisão: df.loc[linhas,colunas]
linhas = clientes.nome.isnull()    - apenas clientes que possuem o nome nulo
colunas = :     - retorna todas as colunas
'''
clientes.loc[clientes.nome.isnull(),:]

,id,nome,sexo,dt_nasc
263,264,NaN,F,7/29/1997
264,265,NaN,M,2/7/1979
268,269,NaN,F,NaN
269,270,NaN,M,NaN


In [15]:
#Mesmo slicing do exemplo anterior, mas retornando dados apenas da coluna nome
clientes.loc[clientes.nome.isnull(),'nome']

263    NaN
264    NaN
268    NaN
269    NaN
Name: nome, dtype: object

In [16]:
#Substitui apenas os valores nulos de nome por 'Sem Nome'
clientes.loc[clientes.nome.isnull(),'nome'] = 'Sem Nome'

In [18]:
#Conferindo se os nulos foram trocados por 'Sem Nome'
clientes[clientes.nome == 'Sem Nome']

,id,nome,sexo,dt_nasc
263,264,Sem Nome,F,7/29/1997
264,265,Sem Nome,M,2/7/1979
268,269,Sem Nome,F,NaN
269,270,Sem Nome,M,NaN


In [19]:
clientes.loc[clientes.sexo.isnull(),'sexo'] = 'o'

In [20]:
clientes.loc[clientes.sexo == 'o']

,id,nome,sexo,dt_nasc
287,288,Victor Ribeiro,o,12/17/1974
290,291,Clara Pinto,o,12/25/1980
294,295,Tiago Lima,o,7/26/1973
301,302,Guilherme Cardoso,o,3/9/1998


In [21]:
clientes.loc[clientes.dt_nasc.isnull(),'dt_nasc'] = '01/12/2023'

In [23]:
clientes.loc[clientes.dt_nasc == '01/12/2023']

,id,nome,sexo,dt_nasc
268,269,Sem Nome,F,01/12/2023
269,270,Sem Nome,M,01/12/2023
276,277,Brenda Santos,F,01/12/2023
282,283,Amanda Barros,F,01/12/2023
